<a href="https://colab.research.google.com/github/Rahulraj31/Diabetic-Retinopathy/blob/main/Diabetic_Retinopathy_224x224_(2019_Data).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import zipfile

local_zip = '/content/drive/MyDrive/Colab Notebooks/archive.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [50]:
import PIL.Image as Image
import cv2
import numpy as np
import pandas as pd

import matplotlib.pylab as plt
%matplotlib inline 
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers,losses
from tensorflow.keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
df= pd.read_csv("/content/train.csv")
df.head()

,id_code,diagnosis
0,000c1434d8d7,2
1,001639a390f0,4
2,0024cdab0c1e,1
3,002c21358ce6,0
4,005b95c28852,0


0 - No_DR
1 - Mild
2 - Moderate
3 - Severe
4 - Proliferate_DR

In [ ]:
from pathlib import Path 
import os
train_data_dir = Path('/content/colored_images')

In [ ]:
train_images_dict = {
    'Mild': list(train_data_dir.glob('Mild/*')) ,   
    'Moderate': list(train_data_dir.glob('Moderate/*')),
    'No_DR': list(train_data_dir.glob('No_DR/*')),
    'Proliferate_DR': list(train_data_dir .glob('Proliferate_DR/*')),
    'Severe': list(train_data_dir.glob('Severe/*')),
}

labels_dict = {
     'Mild': 'mild' ,   
    'Moderate':'moderate',
    'No_DR': 'no_dr',
    'Proliferate_DR':'proliferate_dr',
    'Severe': 'sever',
}

In [ ]:
img= cv2.imread(str(train_images_dict['Mild'][0]))
img.shape

(224, 224, 3)

In [ ]:
X, y = [], []
for name,images in train_images_dict.items():#returns key , path for each record
    for image in images:
        img=str(image)

        X.append(img)
        y.append(labels_dict[name])

In [ ]:
Fseries = pd.Series(X, name="filepaths",dtype=str)
Lseries = pd.Series(y, name="labels",dtype=str)
data = pd.concat([Fseries,Lseries], axis=1)
dr_df = pd.DataFrame(data)

print(dr_df.head())
print("---------------------")
print(dr_df["labels"].value_counts())


                                       filepaths labels
0  /content/colored_images/Mild/5777ef74c9ec.png   mild
1  /content/colored_images/Mild/a28bfb772f50.png   mild
2  /content/colored_images/Mild/5cab3ef4b31c.png   mild
3  /content/colored_images/Mild/5633ced07d8e.png   mild
4  /content/colored_images/Mild/9782c0489eca.png   mild
---------------------
no_dr             1805
moderate           999
mild               370
proliferate_dr     295
sever              193
Name: labels, dtype: int64


In [83]:
image_gen = ImageDataGenerator(rescale=1.00/255,validation_split=0.2,vertical_flip=True
                               )

train = image_gen.flow_from_dataframe(dataframe= dr_df,x_col="filepaths",y_col="labels",
                                      target_size=(224,224),
                                      color_mode='rgb',
                                      class_mode="categorical",
                                      batch_size=8,
                                      shuffle=False,
                                      subset="training",
                         
                                     )

val = image_gen.flow_from_dataframe(dataframe= dr_df,x_col="filepaths",y_col="labels",
                                       target_size=(224,224),
                                      color_mode='rgb',
                                      class_mode="categorical",
                                      batch_size=8,
                                      subset="validation",
                                      shuffle=False          
                                     )


Found 2930 validated image filenames belonging to 5 classes.
Found 732 validated image filenames belonging to 5 classes.


In [84]:
classes=list(train.class_indices.keys())
print (classes)

['mild', 'moderate', 'no_dr', 'proliferate_dr', 'sever']


In [85]:
from tensorflow.keras.applications import VGG16

cnn_base = VGG16(include_top = False,
                 weights = 'imagenet',
                 input_shape = (224, 224, 3),
                 classes = 5,
                 classifier_activation = 'softmax')

cnn_base.trainable = False

In [86]:
pretrainedCNN_model = Sequential([
                                     cnn_base,
                                  layers.Flatten(),
                                  layers.Dense(1000, activation = 'relu'),
                                  layers.Dropout(0.25),
                                  layers.Dense(500, activation = 'relu'),
                                  layers.Dense(100, activation = 'relu'),
                                  layers.Dense(5, activation = 'softmax')  ,  
])

In [87]:
pretrainedCNN_model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [88]:
pretrainedCNN_model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Functional)           (None, 7, 7, 512)         14714688  
_________________________________________________________________
flatten_5 (Flatten)          (None, 25088)             0         
_________________________________________________________________
dense_15 (Dense)             (None, 1000)              25089000  
_________________________________________________________________
dropout_5 (Dropout)          (None, 1000)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 500)               500500    
_________________________________________________________________
dense_17 (Dense)             (None, 100)               50100     
_________________________________________________________________
dense_18 (Dense)             (None, 5)                

In [89]:
History = pretrainedCNN_model.fit(train, validation_data=val, epochs=10,verbose=1,
                                  )

Epoch 1/10
367/367 [==============================] - 26s 68ms/step - loss: 1.2084 - accuracy: 0.6843 - val_loss: 5.5113 - val_accuracy: 0.4604
Epoch 2/10
367/367 [==============================] - 25s 69ms/step - loss: 0.6806 - accuracy: 0.7645 - val_loss: 6.3535 - val_accuracy: 0.4399
Epoch 3/10
367/367 [==============================] - 25s 67ms/step - loss: 0.6025 - accuracy: 0.7782 - val_loss: 3.6398 - val_accuracy: 0.4522
Epoch 4/10
367/367 [==============================] - 25s 67ms/step - loss: 0.5690 - accuracy: 0.7863 - val_loss: 6.2038 - val_accuracy: 0.4249
Epoch 5/10
367/367 [==============================] - 25s 67ms/step - loss: 0.5665 - accuracy: 0.7843 - val_loss: 5.3962 - val_accuracy: 0.4686
Epoch 6/10
367/367 [==============================] - 25s 68ms/step - loss: 0.5562 - accuracy: 0.7863 - val_loss: 5.4252 - val_accuracy: 0.4672
Epoch 7/10
367/367 [==============================] - 25s 67ms/step - loss: 0.5478 - accuracy: 0.7805 - val_loss: 5.5310 - val_accuracy: